In [1]:
import gym_unbalanced_disk
import gym
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm #a neat progress bar

In [2]:
env = gym_unbalanced_disk.UnbalancedDisk_exp(umax = 2.5,dt = 0.025)

def get_exp(Nsamps,q,f0,umax=3.0,seed=46):
    print(q,f0)
    np.random.seed(seed)
    from scipy import signal
    u0 = np.random.uniform(-1,1,size=Nsamps)
    if f0<1:
        u1 = signal.lfilter(*signal.butter(6,f0),u0)
    else:
        u1 = u0
    
    u1 /= max(np.max(u1),-np.min(u1))
    
    return umax*abs(u1)**(1/q)*np.sign(u1) if np.isfinite(q) else umax*np.sign(u1)

def apply_exp(ulist):
    obs = env.reset()
    ulist2 = []
    ylist = []
    for u in tqdm(ulist):
        ylist.append(obs)
        if obs[0]>np.pi*3/4: # a bit of a controller to restrict going over the top
            u = -env.unwrapped.umax
        if obs[0]<-np.pi*3/4:
            u = env.unwrapped.umax
        obs, reward, done, info = env.step(u)
        ulist2.append(u)
    env.reset()
    return np.array(ylist), np.array(ulist2)

starting matlab engine...done
connecting to experimental setup...done


In [3]:
ulist = get_exp(10000,q=4,f0=0.3,umax=2.5)
ylist,ulist = apply_exp(ulist)

from scipy.io import savemat

save_dic = dict(ulist=ulist,ylist=ylist)

np.savez('UnbalancedDisk-exp-all-data-Maarten.npz',**save_dic)
savemat('UnbalancedDisk-exp-all-data-Maarten.mat', save_dic)
save_dic = dict(ulist=ulist,ylist=ylist[:,0])

np.savez('UnbalancedDisk-exp-data.npz-Maarten',**save_dic)
savemat('UnbalancedDisk-exp-data-Maarten.mat', save_dic)

4 0.3
